In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_train = pd.read_csv('urop_dataset_training.csv')
df_val = pd.read_csv('urop_dataset_validation.csv')

In [3]:
df_train = df_train[df_train["Scan included on RECIST form? (y/n)"] == "yes"]

In [3]:
# this field is the class we're trying to predict and hence we have to strip any whitespaces from it
df_train["Objective Response per RECIST v1.1"] = df_train["Objective Response per RECIST v1.1"].apply(lambda x: x.strip())

In [ ]:
df_train.groupby(['Treatment setting']).size()

In [ ]:
df_val.groupby(['Treatment setting']).size()

In [ ]:
df_train.groupby(['Objective Response per RECIST v1.1']).size()

In [ ]:
df_train.groupby(['Type of scan']).size()

In [ ]:
df_val.groupby(['Type of scan']).size()

In [ ]:
df_train.groupby(['Scan timepoint (baseline = prior to treatment start, ontx = during treatment or prior to progression if stopped treatment , progression = time of RECIST defined progression)']).size()

In [10]:
by_patient = df_train.groupby(['Patient ID']).first()

In [ ]:
by_patient.groupby(['Objective Response per RECIST v1.1']).size()

In [ ]:
print(df_train["Scan report text"][20])

In [5]:
findings = df_train["Scan report text"].apply(lambda x: True if x.find("FINDING") != -1 else False)
findings[findings == False]

Series([], Name: Scan report text, dtype: bool)

In [6]:
findings = df_train["Scan report text"].apply(lambda x: True if x.find("IMPRESSION") != -1 else False)
findings[findings == False]

Series([], Name: Scan report text, dtype: bool)

In [23]:
def isolate_finding(x):
    start = x.find("FINDING")
    start = x.find("\n", start)
    stop = x.find("IMPRESSION")
    return x[start: stop]

df_train["Findings"] = df_train["Scan report text"].apply(lambda x: isolate_finding(x))

In [8]:
occurences = []
for i in only_findings:
    occurences.extend(re.findall('([A-Z]+)(\s+[A-Z]+)*[:]', i))

In [9]:
len(occurences)

18945

In [10]:
part_counts = Counter(occurences)

In [ ]:
part_counts

In [ ]:
for i in range(len(occurences)):
    occurences[i] = occurences[i][0] + occurences[i][1]
occurences

In [28]:
numbers = []
for i in only_findings:
    #print(i)
    numbers.extend(re.findall('[0-9]*\.?[0-9]+\s*(?:cm|m|mm)?\s*x\s*[0-9]*\.?[0-9]+', i))
    
df_train["numbers"] = df_train["Findings"].apply(lambda x: re.findall('[0-9]*\.?[0-9]+\s*(?:cm|m|mm)?\s*x\s*[0-9]*\.?[0-9]+', x))
    
# by instead of x
# [0-9]*\.?[0-9]+\s*(?:cm|m|mm)?\s*(x|by)\s*[0-9]*\.?[0-9]+

In [29]:
df_train["numbers"][0]

['5.1 x 4', '5.2 x 3.8', '1.5 x 0.8', '1.4 x 1', '1.1 x 1']